In [49]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jkuma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:

import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jkuma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [51]:
data=pd.read_csv(r"C:\Users\Jkuma\Downloads\tourism_India.csv")

In [73]:
data.head()

,Unnamed: 0,Place,Rating,Best Time to Visit,Description,processed_description
0,0,Manali,4.5,October to Jun,"With spectacular valleys, breathtaking views, ...",spectacular valley breathtak view snowcap moun...
1,1,Leh Ladakh,4.6,Jun to Sep,Ladakh is a union territory in the Kashmir reg...,ladakh union territori kashmir region india fo...
2,2,Coorg,4.2,October to March,Located amidst imposing mountains in Karnataka...,locat amidst impos mountain karnataka perpetu ...
3,3,Andaman,4.5,October to Jun,Replete with turquoise blue water beaches and ...,replet turquois blue water beach bit histori a...
4,4,Lakshadweep,4.0,September to May,"Formerly known as Laccadive Islands, Lakshadwe...",formerli known laccad island lakshadweep trans...


In [74]:
data.tail()

,Unnamed: 0,Place,Rating,Best Time to Visit,Description,processed_description
95,95,Pushkar,4.0,October to March,Pushka is a small temple town located around 1...,pushka small templ town locat around km ajmer ...
96,96,Chittorgarh,4.1,October to March,"Located in South-Eastern Rajasthan, Chittorgar...",locat rajasthan chittorgarh known chittorgarh ...
97,97,Nahan,3.9,April to September,"Away from the buzzing crowd of the city, Nahan...",away buzz crowd citi nahan perfect romant geta...
98,98,Lavasa,3.9,hroughout the year,"Known as India's newest hill station, the Lava...",known india newest hill station lavasa corpor ...
99,99,Poovar,4.3,August to March,Poovar is a small rustic town situated 27 kms ...,poovar small rustic town situat km thiruvanant...


In [75]:
# Preprocess data
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

In [76]:
def preprocess(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [porter.stem(token) for token in tokens if token.isalpha() and token not in stop_words]
    return ' '.join(tokens)

data['processed_description'] = data['Description'].apply(preprocess)


In [77]:
# Create TF-IDF matrix
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['processed_description'])


In [78]:
# Compute cosine similarity
similarity_scores = cosine_similarity(tfidf_matrix)

In [79]:
# Define function to recommend destinations based on user input
def recommend_destinations(rating, best_time, user_description, num_recommendations, description_vectors):
    # Filter destinations based on rating and best time
    filtered_data = data[(data['Rating'] >= rating) &
                         (data['Best Time to Visit'] == best_time)]
    
    # Vectorize user description
    user_description_vector = tfidf.transform([user_description])
    
    # Calculate cosine similarity between user description and destination descriptions
    similarity_scores = cosine_similarity(user_description_vector, tfidf_matrix)
    
    # Get indices of top recommended destinations
    top_indices = similarity_scores.argsort()[0][-num_recommendations:][::-1]
    
    # Return recommended destinations
    recommendations = data['Place'].iloc[top_indices].tolist()
    return recommendations

In [84]:
# Test the function
place = 'Manali'
rating = 4.5
best_time = 'October to Jun'
user_description = 'mountains'
num_recommendations = 5

In [85]:
recommended_destinations = recommend_destinations(rating, best_time, user_description, num_recommendations,tfidf_matrix )
print(f"Recommended destinations based on your description:")
for destination in recommended_destinations:
    print(destination)


Recommended destinations based on your description:
Poovar
Pondicherry
Agra
Kolkata
Jodhpur


In [87]:
# Test the function with different inputs
rating = 4.2
best_time = 'October to March'
user_description = 'Peaceful surroundings and coffee plantations'
num_recommendations = 3

recommended_destinations = recommend_destinations(rating, best_time, user_description, num_recommendations,tfidf_matrix)
print(f"Recommended destinations based on your description:")
for destination in recommended_destinations:
    print(destination)


Recommended destinations based on your description:
Poovar
Pondicherry
Agra


In [88]:
# Print recommended destinations
for i, indices in enumerate(top_indices):
    print(f"\nRecommendations for User {i+1}:")
    for idx in indices:
        print(f"- Destination: {processed_description[idx]}")


Recommendations for User 1:


NameError: name 'processed_description' is not defined

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# User input with descriptions and corresponding place names
user_data = [
    {
        'place': 'Beach Destination',
        'description': "I love beaches and water sports."
    },
    {
        'place': 'Nature Retreat',
        'description': "I enjoy hiking and being close to nature."
    },
    {
        'place': 'Historical City',
        'description': "I'm interested in historical landmarks and architecture."
    },
    {
        'place': 'City with Nightlife',
        'description': "I prefer vibrant cities with a lively nightlife."
    }
]

# Destination descriptions
destination_data = [
    {
        'place': 'Destination A',
        'description': "This destination offers beautiful beaches and a wide range of water sports activities."
    },
    {
        'place': 'Destination B',
        'description': "It is surrounded by stunning natural landscapes and offers numerous hiking trails."
    },
    {
        'place': 'Destination C',
        'description': "This city is known for its rich history and impressive architectural landmarks."
    },
    {
        'place': 'Destination D',
        'description': "Experience the vibrant nightlife of this bustling city with its numerous clubs and entertainment options."
    }
]

# Preprocess and vectorize the descriptions
vectorizer = TfidfVectorizer()
user_descriptions = [data['description'] for data in user_data]
user_vectors = vectorizer.fit_transform(user_descriptions)
destination_descriptions = [data['description'] for data in destination_data]
destination_vectors = vectorizer.transform(destination_descriptions)

# Calculate cosine similarity between user and destination vectors
similarity_scores = cosine_similarity(user_vectors, destination_vectors)

# Recommend destinations based on highest similarity score
num_recommendations = 3  # Number of recommendations to provide
top_indices = similarity_scores.argsort()[:, -num_recommendations:][::-1]

# Print recommended destinations with place names
for i, indices in enumerate(top_indices):
    print(f"\nRecommendations for User {i+1}:")
    for idx in indices:
        place_name = destination_data[idx]['place']
        print(f"- Destination: {place_name}\n  Description: {destination_descriptions[idx]}")



Recommendations for User 1:
- Destination: Destination B
  Description: It is surrounded by stunning natural landscapes and offers numerous hiking trails.
- Destination: Destination C
  Description: This city is known for its rich history and impressive architectural landmarks.
- Destination: Destination D
  Description: Experience the vibrant nightlife of this bustling city with its numerous clubs and entertainment options.

Recommendations for User 2:
- Destination: Destination D
  Description: Experience the vibrant nightlife of this bustling city with its numerous clubs and entertainment options.
- Destination: Destination B
  Description: It is surrounded by stunning natural landscapes and offers numerous hiking trails.
- Destination: Destination C
  Description: This city is known for its rich history and impressive architectural landmarks.

Recommendations for User 3:
- Destination: Destination D
  Description: Experience the vibrant nightlife of this bustling city with its num

In [ ]:
def recommend_destinations(place, rating, best_time,user_description,num_recommendations):
    idx = data.index[(data['Place'] == place) & (data['Rating'] >= rating) & (data['Best Time to Visit'] == best_time)].tolist()[0]
    sim_scores = list(enumerate(similarity_scores[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    destination_indices = [i[0] for i in sim_scores]
    return data['Place'].iloc[destination_indices]